In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import dask.dataframe as dd
from geopy.distance import geodesic
import string

import warnings
warnings.filterwarnings('ignore')

In [73]:
dir = '../csv/'
fname = 'credit_card_fraud_sample_2019_Dec_100K'
df = pd.read_csv(dir+fname+'.csv')

#df = pd.read_csv('../csv/credit_card_fraud_2019_q1_15K_rows.csv')
#df = pd.read_csv('../csv/credit_card_fraud_2019_Dec.csv')
#df = pd.read_csv('../csv/credit_card_fraud_sample_2019_Dec_100K.csv')



In [74]:
df = df.drop(columns=['quarter']) # quarter added while spliting the larger dataset
df = df.rename(columns = {'Unnamed: 0':'id'})
df.head()

,id,ssn,cc_num,first,last,gender,street,city,state,zip,...,trans_num,trans_date,trans_time,unix_time,category,amt,is_fraud,merchant,merch_lat,merch_long
0,18832098,087-12-2118,639017474813,Zachary,Parks,M,94238 Jacob Cliff Apt. 458,Columbia,MO,65203,...,a36eecc709911793fe140ba7d29404ca,2019-12-26,06:36:03,1577322363,entertainment,3.65,0,"fraud_Turcotte, McKenzie and Koss",37.978633,-91.813002
1,24801519,267-50-6963,6567701540795312,Jeremy,Soto,M,29865 Bush Centers,Goldendale,WA,98620,...,5c0acce002c0e0b56de66d9de0cbff80,2019-12-28,22:53:34,1577553814,shopping_pos,5.25,0,fraud_Hermann and Sons,45.829641,-121.389115
2,31141574,411-62-3428,180022197664580,Samuel,Mcdonald,M,4570 Ruth Harbors,Palo Alto,CA,94306,...,c363df118159e47c9a46ff906471c0c0,2019-12-23,13:23:24,1577087604,shopping_pos,25.31,0,fraud_Conroy Ltd,37.825064,-121.644324
3,25784982,841-48-6189,3584095374557286,Nicholas,Davis,M,83759 Shannon Meadows,Kohler,WI,53044,...,e33a7e2168a80e7b3a9c7ba239219259,2019-12-03,09:21:51,1575345111,shopping_pos,95.28,0,fraud_Hudson-Grady,43.686522,-86.947295
4,19821729,603-28-2907,6011226654586127,Mark,Sanchez,M,21975 Pam Garden,Birmingham,MI,48009,...,bf6116234aaf567b13a629cf6b604b0b,2019-12-27,21:56:44,1577464004,shopping_pos,73.33,0,fraud_Dooley Inc,43.031478,-83.903585


In [75]:
print('Shape: ', df.shape)
print('Columns: ', df.columns)
print('Data types: ', df.info())    



Shape:  (100000, 27)
Columns:  Index(['id', 'ssn', 'cc_num', 'first', 'last', 'gender', 'street', 'city',
       'state', 'zip', 'lat', 'long', 'city_pop', 'job', 'dob', 'acct_num',
       'profile', 'trans_num', 'trans_date', 'trans_time', 'unix_time',
       'category', 'amt', 'is_fraud', 'merchant', 'merch_lat', 'merch_long'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 27 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id          100000 non-null  int64  
 1   ssn         100000 non-null  object 
 2   cc_num      100000 non-null  int64  
 3   first       100000 non-null  object 
 4   last        100000 non-null  object 
 5   gender      100000 non-null  object 
 6   street      100000 non-null  object 
 7   city        100000 non-null  object 
 8   state       100000 non-null  object 
 9   zip         100000 non-null  int64  
 10  lat         100000 non-null

In [76]:
df.isna().sum()


id            0
ssn           0
cc_num        0
first         0
last          0
gender        0
street        0
city          0
state         0
zip           0
lat           0
long          0
city_pop      0
job           0
dob           0
acct_num      0
profile       0
trans_num     0
trans_date    0
trans_time    0
unix_time     0
category      0
amt           0
is_fraud      0
merchant      0
merch_lat     0
merch_long    0
dtype: int64

In [78]:
#convert to proper datetime
date_cols = ['dob', 'trans_date']
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce') # will use only dates
df['trans_time'] = pd.to_datetime(df['trans_time'], format='%H:%M:%S', errors='coerce') # will use only time

#Convert numericals
df['zip'] = df['zip'].astype(int)
df['amt'] = df['amt'].astype(float)
df['is_fraud'] = df['is_fraud'].astype(int)
df['unix_time'] = df['unix_time'].astype(int)
#df['quarter'] = df['quarter'].astype(int)

#Clean leading and trailing whitespaces
cat_columns = df.select_dtypes(include = ['object']).columns    
for col in cat_columns:
    if df[col].dtype == 'object':
        df['col'] = df[col].astype(str).str.strip()
    else:
        df[col] = df[col].str.strip()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   id          100000 non-null  int64         
 1   ssn         100000 non-null  object        
 2   cc_num      100000 non-null  int64         
 3   first       100000 non-null  object        
 4   last        100000 non-null  object        
 5   gender      100000 non-null  object        
 6   street      100000 non-null  object        
 7   city        100000 non-null  object        
 8   state       100000 non-null  object        
 9   zip         100000 non-null  int32         
 10  lat         100000 non-null  float64       
 11  long        100000 non-null  float64       
 12  city_pop    100000 non-null  int64         
 13  job         100000 non-null  object        
 14  dob         100000 non-null  datetime64[ns]
 15  acct_num    100000 non-null  int64         
 16  pro

In [79]:
print(df['gender'].value_counts())
df['gender'] = df['gender'].map({'M': 1, 'F': 0})
print(df['gender'].value_counts())

gender
F    51761
M    48239
Name: count, dtype: int64
gender
0    51761
1    48239
Name: count, dtype: int64


In [ ]:
# transaction amount, 
print(df['amt'].describe())
#just in case, clip negative values, may change on different datasets, also check for outliers later
df['amt'] = df['amt'].clip(lower=0)
print(df['amt'].describe())

count    100000.000000
mean         67.459051
std         164.191618
min           1.000000
25%           7.490000
50%          34.500000
75%          79.280000
max       11078.860000
Name: amt, dtype: float64
count    100000.000000
mean         67.459051
std         164.191618
min           1.000000
25%           7.490000
50%          34.500000
75%          79.280000
max       11078.860000
Name: amt, dtype: float64


In [81]:
#trans_num should be overal unique, but check for duplicates, if any consider removing them
duplicates = df['trans_num'].duplicated(keep=False)
print('Duplicate transactions: ', duplicates.sum())

Duplicate transactions:  0


In [82]:
#clean merchant names
df['merchant'] = df['merchant'].str.lower()
df['merchant'] = df['merchant'].str.replace('fraud_', '')
df['merchant'] = df['merchant'].str.replace('merch_', '')
df['merchant'] = df['merchant'].str.replace('merch', '')


In [83]:
#remove only exact duplicate rows
df = df.drop_duplicates()

In [84]:
# Ensure latitude is between -90 and 90
df['lat'] = df['lat'].clip(-90, 90)
df['merch_lat'] = df['merch_lat'].clip(-90, 90)

# Ensure longitude is between -180 and 180
df['long'] = df['long'].clip(-180, 180)
df['merch_long'] = df['merch_long'].clip(-180, 180)


In [85]:
df['state'] = df['state'].str.upper()
print(df['state'].value_counts())

# Define region lists
NORTHEAST = ['CT', 'ME', 'MA', 'NH', 'RI', 'VT', 'NJ', 'NY', 'PA']
MIDWEST = ['IL', 'IN', 'IA', 'KS', 'MI', 'MN', 'MO', 'NE', 'ND', 'OH', 'SD', 'WI']
SOUTH = ['AL', 'AR', 'DE', 'FL', 'GA', 'KY', 'LA', 'MD', 'MS', 'NC', 'OK', 'SC', 'TN', 'TX', 'VA', 'WV', 'DC']
WEST = ['AK', 'AZ', 'CA', 'CO', 'HI', 'ID', 'MT', 'NV', 'NM', 'OR', 'UT', 'WA', 'WY']

# Create the region dictionary
REGION_DICT = {state: 'Northeast' for state in NORTHEAST}
REGION_DICT.update({state: 'Midwest' for state in MIDWEST})
REGION_DICT.update({state: 'South' for state in SOUTH})
REGION_DICT.update({state: 'West' for state in WEST})

# Define the function to map state to region
def get_region(state):
    if not isinstance(state, str):
        return 'Unknown'
    state = state.strip().upper()
    return REGION_DICT.get(state, 'Unknown')


df['region'] = df['state'].apply(get_region)
print(df['region'].value_counts())

state
CA    11658
TX     7745
NY     6471
FL     5828
IL     4589
PA     4352
OH     3584
GA     3280
NC     3187
MI     3156
NJ     2802
VA     2741
AZ     2307
WA     2256
TN     2028
IN     2028
MA     2000
MD     1923
MO     1818
MN     1772
SC     1767
WI     1662
CO     1523
KY     1424
AL     1373
LA     1356
OR     1215
OK     1202
CT     1183
NV     1050
IA     1000
AR      960
UT      923
MS      863
KS      807
NM      741
NE      604
WV      549
HI      499
ME      473
ID      462
NH      448
DE      339
VT      330
SD      321
MT      313
RI      288
AK      236
ND      221
WY      174
DC      169
Name: count, dtype: int64
region
South        36734
West         23357
Midwest      21562
Northeast    18347
Name: count, dtype: int64


In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 29 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   id          100000 non-null  int64         
 1   ssn         100000 non-null  object        
 2   cc_num      100000 non-null  int64         
 3   first       100000 non-null  object        
 4   last        100000 non-null  object        
 5   gender      100000 non-null  int64         
 6   street      100000 non-null  object        
 7   city        100000 non-null  object        
 8   state       100000 non-null  object        
 9   zip         100000 non-null  int32         
 10  lat         100000 non-null  float64       
 11  long        100000 non-null  float64       
 12  city_pop    100000 non-null  int64         
 13  job         100000 non-null  object        
 14  dob         100000 non-null  datetime64[ns]
 15  acct_num    100000 non-null  int64         
 16  pro

### Narrowing some categories

In [87]:
def assign_time_segment(hour):
    if 0 <= hour < 6:
        return 'Late_Night_0-6'
    elif 6 <= hour < 12:
        return 'Morning_6-12'
    elif 12 <= hour < 18:
        return 'Afternoon_12-18'
    else:
        return 'Evening_18-24'

def assign_age_group(age):
    if age < 15:
        return 'child'
    elif age < 20:
        return 'teenager'
    elif age < 40:
        return 'young_adult'
    elif age < 60:
        return 'middle_aged'
    elif age < 80:
        return 'senior'
    else:
        return 'elderly'

#df['trans_date'] = pd.to_datetime(df['trans_date'])
#df['trans_time'] = pd.to_datetime(df['trans_time'])

#df['trans_year'] = df['trans_date'].dt.year
#df['trans_month'] = df['trans_date'].dt.month
#df['trans_day'] = df['trans_date'].dt.day

#df['trans_hour'] = df['trans_time'].dt.hour
#df['trans_minute'] =df['trans_time'].dt.minute
#df['trans_second'] =df['trans_time'].dt.second
df['trans_time_segment'] = df['trans_time'].dt.hour.apply(assign_time_segment)

#df['dob'] = pd.to_datetime(df['dob'])
#df['age'] = df['trans_year']  - df['dob'].dt.year

#add an age column at the time of the transaction
age_df = df['trans_date'].dt.year - pd.to_datetime(df['dob']).dt.year

df['age_group'] = age_df.apply(assign_age_group)

print('age distribution', age_df.describe()) #check for outliers...
print(df['age_group'].value_counts())

age distribution count    100000.000000
mean         40.663180
std          17.850472
min          11.000000
25%          27.000000
50%          38.000000
75%          52.000000
max          91.000000
dtype: float64
age_group
young_adult    43480
middle_aged    30712
senior         12303
teenager        8648
elderly         3703
child           1154
Name: count, dtype: int64


In [88]:
df['category'].value_counts()

category
shopping_pos      79227
entertainment     13845
shopping_net       6742
grocery_pos          74
misc_net             28
gas_transport        24
food_dining          12
misc_pos             12
grocery_net           8
travel                7
home                  7
kids_pets             6
personal_care         6
health_fitness        2
Name: count, dtype: int64

In [89]:
#narrow down the CC categories

def get_cc_type(cc_num):
    # Handle missing values (e.g., None or NaN)
    if pd.isna(cc_num):
        return 'Unknown'
    
    # Convert input to string (handles integers, floats, etc.)
    cc_num = str(cc_num)
    
    # Remove non-digit characters (e.g., spaces, hyphens)
    cc_num = ''.join(filter(str.isdigit, cc_num))
    
    # Return 'Unknown' if the result is empty
    if not cc_num:
        return 'Unknown'
    
    # Check UnionPay: starts with 62
    if len(cc_num) >= 2 and cc_num[:2] == '62':
        return 'UnionPay'
    
    # Check Maestro: starts with 50, 56-69
    elif len(cc_num) >= 2 and (cc_num[:2] == '50' or ('56' <= cc_num[:2] <= '69')):
        return 'Maestro'
    
    # Check RuPay: starts with 60, 6521, 6522
    elif len(cc_num) >= 2 and cc_num[:2] == '60':
        return 'RuPay'
    elif len(cc_num) >= 4 and cc_num[:4] in ['6521', '6522']:
        return 'RuPay'
    
    # Check Laser: starts with 6304, 6706, 6709, 6771
    elif len(cc_num) >= 4 and cc_num[:4] in ['6304', '6706', '6709', '6771']:
        return 'Laser'
    
    # Check Solo: starts with 6334, 6767
    elif len(cc_num) >= 4 and cc_num[:4] in ['6334', '6767']:
        return 'Solo'
    
    # Check Switch: starts with 4903, 4905, 4911, 4936, 564182, 633110, 6333, 6759
    elif len(cc_num) >= 4 and cc_num[:4] in ['4903', '4905', '4911', '4936', '6333', '6759']:
        return 'Switch'
    elif len(cc_num) >= 6 and cc_num[:6] in ['564182', '633110']:
        return 'Switch'
    
    # Check JCB: starts with 3528 to 3589
    elif len(cc_num) >= 4 and '3528' <= cc_num[:4] <= '3589':
        return 'JCB'
    
    # Check Diners Club: starts with 300-305, 36, 38, 54, or 55
    elif len(cc_num) >= 3 and ('300' <= cc_num[:3] <= '305'):
        return 'Diners_Club'
    elif len(cc_num) >= 2 and cc_num[:2] in ['36', '38', '54', '55']:
        return 'Diners_Club'
    
    # Check Visa: starts with 4
    elif len(cc_num) >= 1 and cc_num[0] == '4':
        return 'Visa'
    
    # Check MasterCard: starts with 51-55 or 2221-2720
    elif len(cc_num) >= 2 and ('51' <= cc_num[:2] <= '55'):
        return 'MasterCard'
    elif len(cc_num) >= 4 and ('2221' <= cc_num[:4] <= '2720'):
        return 'MasterCard'
    
    # Check American Express: starts with 34 or 37
    elif len(cc_num) >= 2 and cc_num[:2] in ['34', '37']:
        return 'American_Express'
    
    # Check Discover: starts with 6011, 644-649, or 65
    elif len(cc_num) >= 4 and cc_num[:4] == '6011':
        return 'Discover'
    elif len(cc_num) >= 3 and ('644' <= cc_num[:3] <= '649'):
        return 'Discover'
    elif len(cc_num) >= 2 and cc_num[:2] == '65':
        return 'Discover'
    
    # If no pattern matches, return 'Unknown'
    return 'Unknown'

df['cc_type'] = df['cc_num'].apply(get_cc_type)
print(df['cc_type'].value_counts()) 

cc_type
Visa                33136
Maestro             16495
Unknown             14467
JCB                  9996
Diners_Club          9397
MasterCard           8259
American_Express     8098
Switch                152
Name: count, dtype: int64


#Save cleaned df to pickle

In [90]:
df.to_pickle(fname + '_prepared.pkl')

#df = pd.read_pickle(fname + '_prepared.pkl')


In [ ]:
### EDA

In [91]:
df = pd.read_pickle(fname + '_prepared.pkl')
df.head()

,id,ssn,cc_num,first,last,gender,street,city,state,zip,...,amt,is_fraud,merchant,merch_lat,merch_long,col,region,trans_time_segment,age_group,cc_type
0,18832098,087-12-2118,639017474813,Zachary,Parks,1,94238 Jacob Cliff Apt. 458,Columbia,MO,65203,...,3.65,0,"turcotte, mckenzie and koss",37.978633,-91.813002,"fraud_Turcotte, McKenzie and Koss",Midwest,Morning_6-12,middle_aged,Maestro
1,24801519,267-50-6963,6567701540795312,Jeremy,Soto,1,29865 Bush Centers,Goldendale,WA,98620,...,5.25,0,hermann and sons,45.829641,-121.389115,fraud_Hermann and Sons,West,Evening_18-24,young_adult,Maestro
2,31141574,411-62-3428,180022197664580,Samuel,Mcdonald,1,4570 Ruth Harbors,Palo Alto,CA,94306,...,25.31,0,conroy ltd,37.825064,-121.644324,fraud_Conroy Ltd,West,Afternoon_12-18,young_adult,Unknown
3,25784982,841-48-6189,3584095374557286,Nicholas,Davis,1,83759 Shannon Meadows,Kohler,WI,53044,...,95.28,0,hudson-grady,43.686522,-86.947295,fraud_Hudson-Grady,Midwest,Morning_6-12,young_adult,JCB
4,19821729,603-28-2907,6011226654586127,Mark,Sanchez,1,21975 Pam Garden,Birmingham,MI,48009,...,73.33,0,dooley inc,43.031478,-83.903585,fraud_Dooley Inc,Midwest,Evening_18-24,teenager,Maestro


## Descriptive